In [1]:
import pandas as pd
import feather
import talib

# Define the file path
eth_usdt_1m_file = '/allah/freqtrade/user_data/data/binance/futures/ETH_USDT_USDT-1m-futures.feather'

# Read the DataFrame from the feather file
eth_usdt_1m = feather.read_dataframe(eth_usdt_1m_file)
eth_usdt_1m_utc8 = eth_usdt_1m.copy()

# Add 8 hours to the 'date' column and simplify for readability
eth_usdt_1m_utc8['date'] = pd.to_datetime(eth_usdt_1m_utc8['date']) + pd.DateOffset(hours=8)

# Format the 'date' column as a string in the desired format
eth_usdt_1m_utc8['date'] = eth_usdt_1m_utc8['date'].dt.strftime('%Y-%m-%d %H:%M')


In [2]:
eth_usdt_1m_utc8

,date,open,high,low,close,volume
0,2019-11-27 15:45,146.00,146.00,146.00,146.00,0.010
1,2019-11-27 15:46,146.00,146.00,146.00,146.00,0.000
2,2019-11-27 15:47,146.00,146.00,146.00,146.00,0.000
3,2019-11-27 15:48,146.00,146.00,146.00,146.00,0.000
4,2019-11-27 15:49,146.00,146.00,146.00,146.00,0.000
...,...,...,...,...,...,...
2045410,2023-10-18 01:55,1567.15,1567.69,1567.00,1567.02,870.637
2045411,2023-10-18 01:56,1567.02,1568.17,1567.01,1567.73,719.155
2045412,2023-10-18 01:57,1567.73,1568.28,1567.73,1567.87,962.843
2045413,2023-10-18 01:58,1567.87,1567.88,1567.24,1567.26,561.324


In [7]:
# Create a list of candlestick patterns to detect
candlestick_patterns = [
    "CDL2CROWS",
    "CDL3BLACKCROWS",
    "CDL3INSIDE",
    "CDL3LINESTRIKE",
    "CDL3OUTSIDE",
    "CDL3STARSINSOUTH",
    "CDL3WHITESOLDIERS",
    "CDLABANDONEDBABY",
    "CDLADVANCEBLOCK",
    "CDLBELTHOLD",
    "CDLBREAKAWAY",
    "CDLCLOSINGMARUBOZU",
    "CDLCONCEALBABYSWALL",
    "CDLCOUNTERATTACK",
    "CDLDARKCLOUDCOVER",
    "CDLDOJI",
    "CDLDOJISTAR",
    "CDLDRAGONFLYDOJI",
    "CDLENGULFING",
    "CDLEVENINGDOJISTAR",
    "CDLEVENINGSTAR",
    "CDLGAPSIDESIDEWHITE",
    "CDLGRAVESTONEDOJI",
    "CDLHAMMER",
    "CDLHANGINGMAN",
    "CDLHARAMI",
    "CDLHARAMICROSS",
    "CDLHIGHWAVE",
    "CDLHIKKAKE",
    "CDLHIKKAKEMOD",
    "CDLHOMINGPIGEON",
    "CDLIDENTICAL3CROWS",
    "CDLINNECK",
    "CDLINVERTEDHAMMER",
    "CDLKICKING",
    "CDLKICKINGBYLENGTH",
    "CDLLADDERBOTTOM",
    "CDLLONGLEGGEDDOJI",
    "CDLLONGLINE",
    "CDLMARUBOZU",
    "CDLMATCHINGLOW",
    "CDLMATHOLD",
    "CDLMORNINGDOJISTAR",
    "CDLMORNINGSTAR",
    "CDLONNECK",
    "CDLPIERCING",
    "CDLRICKSHAWMAN",
    "CDLRISEFALL3METHODS",
    "CDLSEPARATINGLINES",
    "CDLSHOOTINGSTAR",
    "CDLSHORTLINE",
    "CDLSPINNINGTOP",
    "CDLSTALLEDPATTERN",
    "CDLSTICKSANDWICH",
    "CDLTAKURI",
    "CDLTASUKIGAP",
    "CDLTHRUSTING",
    "CDLTRISTAR",
    "CDLUNIQUE3RIVER",
    "CDLUPSIDEGAP2CROWS",
    "CDLXSIDEGAP3METHODS",
]

# Create a dictionary to map pattern functions to pattern names
pattern_functions = {pattern: getattr(talib, pattern) for pattern in candlestick_patterns}

# Create a list to store the identified patterns
identified_patterns = []

# Loop through the DataFrame and apply the pattern recognition functions
for i in range(max([f.lookback for f in pattern_functions.values()]), len(eth_usdt_1m_utc8)):
    open_prices = eth_usdt_1m_utc8['open'].iloc[i - max([f.lookback for f in pattern_functions.values()]):i + 1].values
    high_prices = eth_usdt_1m_utc8['high'].iloc[i - max([f.lookback for f in pattern_functions.values()]):i + 1].values
    low_prices = eth_usdt_1m_utc8['low'].iloc[i - max([f.lookback for f in pattern_functions.values()]):i + 1].values
    close_prices = eth_usdt_1m_utc8['close'].iloc[i - max([f.lookback for f in pattern_functions.values()]):i + 1].values

    # Iterate through the patterns and check for each one
    for pattern_name, pattern_function in pattern_functions.items():
        pattern_result = pattern_function(open_prices, high_prices, low_prices, close_prices)
        if pattern_result[-1] > 0:
            identified_patterns.append((eth_usdt_1m_utc8['date'].iloc[i], pattern_name))

# Display the identified patterns
for date, pattern in identified_patterns:
    print(f"At {date}, identified {pattern} pattern.")


AttributeError: 'function' object has no attribute 'lookback'